# Opt-in/Opt-out module

## Summary
The purpose of this code is to provide a framework for awaybot to respect opt-in and opt-out requirements for the users from the different teams/channels that awaybot participates in.

## Expected functionality
* Opt-out users have their authored messages filtered out of the awaybot pipeline (before entering our kafka queue)
* __Current functionality assumes opt-in for all users, but opt-in code exists for flexibility.__

## ToDo
* Create config file for test teams
* Write function for on-the-fly opt-in
* Write function for on-the-fly opt-out

In [1]:
%%writefile config.json
{
    "teams" : {
        "slack-pack" : {
            "opt-out" : [
                {"id" : "U2C5M1L6L"}
            ]
        }
    }
}

Overwriting config.json


In [3]:
#######################################
#
# The below need to be populted for bot
# to function
#
#######################################
%env SLACK_BOT_TOKEN=#SLACK_BOT_TOCKEN#

In [5]:
import json

with open("config.json") as f:
    data = json.load(f)
    
    # loop through the teams where k is the team name
    # and v is the dictionary containing opt-out user ids
    for k, v in data["teams"].iteritems():
        print "Accessing team: " + k
        print "Accessing opt-out user_name list: "
        
        # loop through the opt-out list, printing out each 
        # user name
        for ou in v["opt-out"]:
            print ou["user_name"]

Accessing team: slack-pack
Accessing opt-out user_name list: 
carlos


In [11]:
%%writefile errors.py

class UnauthError(Exception):
    """
    Exception raised for error encountered for invalid slack credentials/token
    
    Attributes:
        message -- explanation of the error
    """
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)

Overwriting errors.py


In [4]:
%%writefile optfilter.py
import time
import os
import json
import logging
from errors import *
from slackclient import SlackClient
from logging.handlers import RotatingFileHandler

# create logger
logger = logging.getLogger('opting_logger')
logger.setLevel(logging.DEBUG)
LOGFILE = "log/opting"

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# create console handler, set level of logging and add formatter
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

# create file handler, set level of logging and add formatter
fh = logging.handlers.TimedRotatingFileHandler(LOGFILE, when='M', interval =1)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
fh.suffix = '%Y-%m-%d_%H-%M-%S.log'

# add handlers to logger
logger.addHandler(ch)
logger.addHandler(fh)

class OptFilter:
    """
    Class for our opting policy implementation
    """
    def __init__(self,
                config_path,
                team_name,
                slack_client_token):
        """
        Constructor for OptFilter
        Parameters
        ----------
        config_path : str
            The path to the configuration JSON file for the OptFilter
        team_name : str
            The name of the team being filtered
        slack_client_token : str
            Token to access slack client for team team_name
        """
        
        # try to open a connection to the Slack API and pull the list of 
        # team members
        try:
            logger.info("""Initializing opting filter with parameters:
            \tconfig_path : %s
            \tteam_name : %s
            """ % (config_path, team_name))
            with open(config_path) as f:
                self.config = json.load(f)
            self.team_name = team_name
            self.slack_client = SlackClient(slack_client_token)
            api_call = self.slack_client.api_call("users.list")
            if api_call.get('ok'):
                logger.info("SlackClient successfully connected for OptFilter object.")
                # retrieve all users so we can find our bot
                self.team_members = api_call.get('members')
        except:
                logger.info("Uh oh, should probably put more robust error handling here!")
                # quiting if we hit this guy to cut our losses
                # note that quit() kills the python kernel...
                quit()
                
    def filtered(self,
                 message):
        """
        Function that outputs binary indicator of whether to filter out message
        
        Parameters:
        ----------
        message : dict
            Dictionary containing Slack message and message metadata
        
        Returns:
        -------
        filtered : bool
            True if message should be filtered from pipeline, false otherwise
        """
        if "user" in message:
            for opt_out_team_member in self.config["teams"][self.team_name]["opt-out"]:
                if message["user"] == opt_out_team_member["id"]:
                    logger.debug("Message has been flagged for filtering")
                    return True            
        else:
            logger.debug("Message has NOT been flagged for filtering")
            return False
        
    def opt_out_user(self,
                     team,
                     user_name):
        """
        Function that introduces a user from a team into the opt-out user group
        
        Parameters:
        ----------
        team : str
            Name of the team for opting-out user
        user_name : str
            user_name for the opting out user
        Returns:
        -------
        None
        """
        logger.info("THIS FUNCTION AINT WRITTEN YET. THANKS FOR PLAYING")
        
    def opt_in_user(self,
                    team,
                    user_name):
        """
        Function that introduces a user from a team into the opt-in user group
        
        Parameters:
        ----------
        team : str
            Name of the team for opting-in user
        user_name : str
            user_name for the opting in user
        Returns:
        -------
        None
        """
        logger.info("THIS FUNCTION AINT WRITTEN YET. THANKS FOR PLAYING")

if __name__ == "__main__":
    try:
        f = OptFilter("config.json",
                      "slack-pack",
                      os.environ.get('SLACK_TOKEN'))
        print f.config
#         token = os.environ.get('SLACK_TOKEN')
        token = 'poop'
        sc = SlackClient(token)
        if sc.rtm_connect():
            while True:
                temp = sc.rtm_read()
                for item in temp:
                    if f.filtered(item):
                        pass
                    else:
                        pass
                time.sleep(1)
        else:
            raise UnauthError("Credentials/token invalid")
    except UnauthError as e:
        print 'UnauthError:', e
        quit()

Writing optfilter.py
